<a href="https://colab.research.google.com/github/taerri/meal-kit-analysis/blob/main/03_1_%EB%B9%85%EC%B9%B4%EC%9D%B8%EC%A6%88_%EB%B0%80%ED%82%A4%ED%8A%B8_LDA_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **토픽모델링**

### **데이터 읽기**

In [ ]:
import pandas as pd

In [ ]:
#데이터 읽기
Corpus_all = pd.read_csv("밀키트 뉴스데이터 전처리_최종.csv", encoding="utf8")
print("데이터 개수 :",len(Corpus_all))
Corpus_all.head()

데이터 개수 : 5506


,Unnamed: 0,키워드
0,0,"재배,채소,제공,웰빙,먹거리,하이원리조트,요리사,셰프들,재배,채소,식재료,활용,손님..."
1,1,"백화점,롯데쇼핑,매출,역성장,실적,롯데쇼핑,백화점,회복세,롯데마트,롯데하이마트,e커..."
2,2,"밀키트,창업,가정,안산시,밀키트,기탁,브랜드,100호,밀키트,창업,브랜드,최단,기간..."
3,3,"수산물,할인,정부,소비,촉진,정부,할인,쿠폰,발행,지원,추가,코로나19,최소화,전통..."
4,4,"소상공인,구독,경제,바우처,제공,소상공인,구독경제,중소벤처기업부,비상,경제,중앙,대..."


### **주요 단어 빈도 확인**

In [ ]:
Corpus_org = Corpus_all["키워드"]
Corpus = [wl.split(",")  for wl in Corpus_org]

In [ ]:
def isHangulOrAlphabet(word):
    return word[0].isalpha() 

In [ ]:
isHangulOrAlphabet("12")

False

In [ ]:
for wl in Corpus[:3]:
  print(wl)

['재배', '채소', '제공', '웰빙', '먹거리', '하이원리조트', '요리사', '셰프들', '재배', '채소', '식재료', '활용', '손님', '웰빙', '먹거리', '제공', '호평', '셰프들', '하이원리조트', '호텔', '셰프', '주인공', '호텔', '셰프', '키친', '오가닉팜', '운영', '고객', '건강', '먹거리', '새싹', '허브', '제공', '고객서비스', '에코', '캠페인', '키친가든', '식재료', '활용', '채소', '허브', '열매', '정원', '키친', '오가닉팜', '하이원리조트', '운암정', '조성', '호텔', '셰프', '구슬땀', '20여종', '채소', '허브', '재배', '재배', '채소', '허브', '호텔', '위치', '식음업장', '식재료', '활용', '하이원리조트', 'PB', '상품', 'GOURMET', '고메', '밀키트', '제품', '이용', '채소', '허브', '농법', '재배', '강원랜드', '사내', '벤처', '자원업사이클TF', '도움', '리조트', '배출', '음식물', '쓰레기', '동애등에', '곤충', '먹이', '활용', '자연', '분해', '퇴비', '지원', '활용', '하이원리조트', '조리', '이태규', '부장', '하이원리조트', '호텔', '손님', '정성', '재배', '식재료', '활용', '음식', '대접', '수요', '음식', '노력']
['백화점', '롯데쇼핑', '매출', '역성장', '실적', '롯데쇼핑', '백화점', '회복세', '롯데마트', '롯데하이마트', 'e커머스', '롯데온', '주력', '계열사', '부진', '매출', '역성장', '구조', '조정', '성과', '영업이익', '444%', '증가', '시장', '롯데쇼핑', '영업', '이익', '75억', '444.7%', '동기', '대비', '444.7%', '공시', '매출액', '3.5%', '감소', '영업이익', '송도롯데몰', '공사

In [ ]:
def Word_count(corpus):
    word_cnt = dict()
    for text in corpus:
        for word in text:
            if word_cnt.get(word):
                word_cnt[word]+=1
            else:
                word_cnt[word]=1
    cnt_df = pd.DataFrame.from_dict(word_cnt, orient='index')
    cnt_df.columns = ["개수"]
    cnt_df = cnt_df.sort_values(["개수"],ascending=False)
    return word_cnt, cnt_df

In [ ]:
Words_cnt, Cnt_df = Word_count(Corpus)

In [ ]:
len(Cnt_df)

63678

In [ ]:
# 저장
Cnt_df.to_csv("주요 단어빈도.csv", encoding="utf-8-sig")

# **3-1 LDA  토픽모델링**

In [ ]:
pip install gensim

In [ ]:
pip install pyLDAvis

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
import pickle
from gensim import corpora
import gensim

In [ ]:
class LDA_model:
    def __init__(self,corpus):
        self.corpus = corpus
        return;
    def mk_detokenized_doc(self):
        self.detokenized_doc = []
        for i in range(len(self.corpus)):
            try:
                t = ' '.join(self.corpus.iloc[i])
                self.detokenized_doc.append(t)
            except:
                print(i)
        return;
    def tf_idf_vector(self,max_features):
        self.vectorizer = TfidfVectorizer(max_features= max_features) # 상위 1000개의 단어 보존
        self.X = self.vectorizer.fit_transform(self.detokenized_doc)
        return;
    def mk_word2id(self):
        self.word2id = defaultdict(lambda : 0)
        for idx, feature in enumerate(self.vectorizer.get_feature_names()):
            self.word2id[feature] = idx
        return;
    def print_word2id(self,n=1):
        for i, sent in enumerate(self.detokenized_doc):
            print('====== document[%d] ======' % i)
            print( [ (token, self.X[i, self.word2id[token]]) for token in sent.split() ] )
            if i==n:
                break
        return;
    def mk_lda_model(self,n_topic=10,max_iter=1):
        self.lda_model=LatentDirichletAllocation(n_components=n_topic,learning_method='online',random_state=777,max_iter=max_iter)
        self.lda_top=self.lda_model.fit_transform(self.X)
        return;
    def print_lda(self):
        print("-------------------------------------")
        print(self.lda_model.components_)
        print(self.lda_model.components_.shape) 
        return;
    def print_topic_lda(self,n=5):
        print("-------------------------------------")
        terms = self.vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.
        for idx, topic in enumerate(self.lda_model.components_):
            print("Topic %d:" % (idx+1), [(terms[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
        return;

In [ ]:
model_all = LDA_model(Corpus)

# **3-2 LDA 토픽모델링 시각화**

- 토픽별 단어 분포

In [ ]:
dictionary = corpora.Dictionary(model_all.corpus)
corpus = [dictionary.doc2bow(text) for text in model_all.corpus]
NUM_TOPICS = 9
num_words = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=num_words)
for topic in topics:
    print(topic)
dictionary.save('dictionary_전처리.gensim')
pickle.dump(corpus, open('corpus_전처리.pkl', 'wb'))
ldamodel.save('gensim_model_전처리.gensim')

(0, '0.043*"밀키트" + 0.020*"시장" + 0.019*"제품" + 0.015*"매출" + 0.014*"간편식" + 0.013*"요리" + 0.010*"메뉴" + 0.010*"출시" + 0.010*"판매" + 0.010*"증가" + 0.008*"브랜드" + 0.008*"식품" + 0.008*"조리" + 0.007*"식재료" + 0.006*"인기" + 0.006*"대비" + 0.006*"이마트" + 0.006*"상품" + 0.006*"가정" + 0.006*"성장" + 0.006*"재료" + 0.005*"소비자" + 0.005*"손질" + 0.005*"피코크" + 0.005*"CJ제일제당" + 0.005*"관계자" + 0.005*"확대" + 0.004*"레시피" + 0.004*"수요" + 0.004*"셰프"')
(1, '0.012*"식품" + 0.009*"사업" + 0.008*"대표" + 0.007*"기업" + 0.007*"창업" + 0.006*"산업" + 0.005*"지역" + 0.005*"지원" + 0.004*"생산" + 0.004*"선정" + 0.004*"분야" + 0.004*"시장" + 0.004*"공간" + 0.004*"제품" + 0.004*"스타트업" + 0.004*"코로나19" + 0.003*"미국" + 0.003*"밀키트" + 0.003*"활용" + 0.003*"식물" + 0.003*"교육" + 0.003*"푸드" + 0.003*"회사" + 0.003*"가치" + 0.003*"한국" + 0.003*"운영" + 0.003*"제조" + 0.003*"현대그린푸드" + 0.003*"중국" + 0.003*"진행"')
(2, '0.013*"행사" + 0.012*"판매" + 0.012*"할인" + 0.008*"온라인" + 0.008*"코로나19" + 0.008*"지역" + 0.007*"진행" + 0.007*"상품" + 0.007*"이마트" + 0.006*"매장" + 0.006*"홈플러스" + 0.006*"밀키트" + 0.005*"점포" + 0.004

In [ ]:
pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.save_html(vis,"lda_전처리.html")

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
len(dictionary) # 각 단어의 정수형 표시

63678

In [ ]:
NUM_TOPICS = 9
num_words = 30
#ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=num_words)
for topic in topics:
    print(topic)

(0, '0.043*"밀키트" + 0.020*"시장" + 0.019*"제품" + 0.015*"매출" + 0.014*"간편식" + 0.013*"요리" + 0.010*"메뉴" + 0.010*"출시" + 0.010*"판매" + 0.010*"증가" + 0.008*"브랜드" + 0.008*"식품" + 0.008*"조리" + 0.007*"식재료" + 0.006*"인기" + 0.006*"대비" + 0.006*"이마트" + 0.006*"상품" + 0.006*"가정" + 0.006*"성장" + 0.006*"재료" + 0.005*"소비자" + 0.005*"손질" + 0.005*"피코크" + 0.005*"CJ제일제당" + 0.005*"관계자" + 0.005*"확대" + 0.004*"레시피" + 0.004*"수요" + 0.004*"셰프"')
(1, '0.012*"식품" + 0.009*"사업" + 0.008*"대표" + 0.007*"기업" + 0.007*"창업" + 0.006*"산업" + 0.005*"지역" + 0.005*"지원" + 0.004*"생산" + 0.004*"선정" + 0.004*"분야" + 0.004*"시장" + 0.004*"공간" + 0.004*"제품" + 0.004*"스타트업" + 0.004*"코로나19" + 0.003*"미국" + 0.003*"밀키트" + 0.003*"활용" + 0.003*"식물" + 0.003*"교육" + 0.003*"푸드" + 0.003*"회사" + 0.003*"가치" + 0.003*"한국" + 0.003*"운영" + 0.003*"제조" + 0.003*"현대그린푸드" + 0.003*"중국" + 0.003*"진행"')
(2, '0.013*"행사" + 0.012*"판매" + 0.012*"할인" + 0.008*"온라인" + 0.008*"코로나19" + 0.008*"지역" + 0.007*"진행" + 0.007*"상품" + 0.007*"이마트" + 0.006*"매장" + 0.006*"홈플러스" + 0.006*"밀키트" + 0.005*"점포" + 0.004

---
# **불러와서 분석**

In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary_전처리.gensim')
corpus = pickle.load(open('corpus_전처리.pkl', 'rb'))
ldamodel = gensim.models.ldamodel.LdaModel.load('gensim_model_전처리.gensim')

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is